In [381]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from settings import settings
from sklearn.utils import resample
from scipy.optimize import minimize
from scipy.stats import multivariate_normal

## Define the search list

In [382]:
search_list = {
    "us": ["us", "hml", 2],
    "developed": ["developed", "hml", 2],
    "emerging": ["emerging", "hml", 2],
    "all": [["us", "developed", "emerging"], "hml", 3],
    "world": ["world", "hml", 2],
    "world_ex_us": ["world_ex_us", "hml", 2],
    "us_mega": ["us", "cmp", 2, "mega"],
    "us_large": ["us", "cmp", 2, "large"],
    "us_small": ["us", "cmp", 2, "small"],
    "us_micro": ["us", "cmp", 2, "micro"],
    "us_nano": ["us", "cmp", 2, "nano"],
}

In [383]:
data_path = Path("result")

In [384]:
regional_pfs_cmp = pd.read_parquet(data_path / "regional_pfs_cmp.parquet")
regional_pfs = pd.read_parquet(data_path / "regional_pfs.parquet")
cluster_labels = pd.read_parquet(data_path / "cluster_labels.parquet")

## Define the empirical Bayes estimation

In [385]:
def eb_prepare(data, scale_alphas, overlapping):
    if overlapping:
        data["obs"] = data.groupby(["region", "characteristic"])["region"].transform(
            "size"
        )
        data = data.loc[data.groupby(["region", "characteristic"])["obs"].idxmax()]
        data = data.drop(columns=["obs"])

    data = data.copy()
    data.loc[:, "ret_neu"] = (
        data.groupby(["region", "characteristic"], group_keys=False)[
            ["ret", "mkt_vw_exc"]
        ]
        .apply(
            lambda x: x["ret"]
            - x["ret"].cov(x["mkt_vw_exc"]) / x["mkt_vw_exc"].var() * x["mkt_vw_exc"]
        )
        .values
    )

    data.loc[:, "ret_neu"] *= 100

    scaling_fct = np.sqrt(10**2 / 12) / data.groupby(["region", "characteristic"])[
        "ret_neu"
    ].transform("std")
    data.loc[:, "ret_neu_scaled"] = data["ret_neu"] * scaling_fct
    data["name_wide"] = data["characteristic"] + "__" + data["region"]

    if scale_alphas:
        data_wide = data.pivot(
            index="eom", columns="name_wide", values="ret_neu_scaled"
        )
    else:
        data_wide = data.pivot(index="eom", columns="name_wide", values="ret_neu")
    return {"long": data, "wide": data_wide}


def block_cluster_func(cor_mat: pd.DataFrame, cl_labels: pd.DataFrame):
    cor_mat = cor_mat.copy()
    cor_mat.index.name = "index"
    cl_labels = cl_labels.copy()

    __cor_long = cor_mat.reset_index().melt(
        id_vars="index", var_name="char2", value_name="cor"
    )
    # char: 요인이름과 region을 분리
    __cor_long[["char2", "region2"]] = __cor_long["char2"].str.split("__", expand=True)
    __cor_long[["char1", "region1"]] = __cor_long["index"].str.split("__", expand=True)

    # 요인별 cluster 이름을 추가
    __cor_long = __cor_long.merge(
        cl_labels[["characteristic", "hcl_label"]].rename(
            columns={"hcl_label": "hcl1"}
        ),
        left_on="char1",
        right_on="characteristic",
        how="left",
    )
    __cor_long = __cor_long.merge(
        cl_labels[["characteristic", "hcl_label"]].rename(
            columns={"hcl_label": "hcl2"}
        ),
        left_on="char2",
        right_on="characteristic",
        how="left",
    )

    # 개별 요인이 포함돼 있는 클러스터와 region을 합침
    __cor_long["hclreg1"] = __cor_long["hcl1"] + "__" + __cor_long["region1"]
    __cor_long["hclreg2"] = __cor_long["hcl2"] + "__" + __cor_long["region2"]

    # Create hcl_pair column

    __cor_long["hcl_pair"] = __cor_long.apply(
        lambda row: "_x_".join(sorted([row["hclreg1"], row["hclreg2"]])), axis=1
    )
    __cor_long["name1"] = __cor_long["char1"] + "__" + __cor_long["region1"]
    __cor_long["name2"] = __cor_long["char2"] + "__" + __cor_long["region2"]

    # 같은 thema안에서 correlation의 평균
    __cluster_wise_cor = (
        __cor_long[__cor_long["name1"] != __cor_long["name2"]]
        .groupby("hcl_pair")["cor"]
        .mean()
        .reset_index(name="cor_avg")
    )
    __cor_long = __cor_long.merge(__cluster_wise_cor, on="hcl_pair", how="left")
    __cor_long["cor_avg"] = np.where(
        __cor_long["name1"] == __cor_long["name2"], 1, __cor_long["cor_avg"]
    )
    __cluster_block_cor_matrix = __cor_long.pivot(
        index="name1", columns="name2", values="cor_avg"
    )
    return __cluster_block_cor_matrix

In [386]:
eb_est = {}
for key, x in search_list.items():
    print(f"Region: {x[0]}")
    regions = x[0]

    # Select the appropriate data
    if x[1] == "cmp":
        base_data = regional_pfs_cmp[regional_pfs_cmp["size_grp"] == x[3]].copy()
    elif x[1] == "hml":
        base_data = regional_pfs.copy()

    if isinstance(regions, str):
        regions = [regions]

    data = base_data[
        (base_data["eom"] >= settings["start_date"])
        & (base_data["eom"] <= settings["end_date"])
        & (base_data["region"].isin(regions))
    ]
    data = eb_prepare(
        data,
        scale_alphas=settings["eb"]["scale_alpha"],
        overlapping=settings["eb"]["overlapping"],
    )
    if x[2] == 3:
        break

Region: us
Region: developed
Region: emerging
Region: ['us', 'developed', 'emerging']


In [387]:
min_obs = settings["eb"]["min_obs"]
fix_alpha = settings["eb"]["fix_alpha"]
bs_cov = settings["eb"]["bs_cov"]
layers = x[2]
shrinkage = settings["eb"]["shrinkage"]
cor_type = settings["eb"]["cor_type"]
bs_samples = settings["eb"]["bs_samples"]
seed = settings["seed"]
sigma = None
priors = None

### Step By Step: Empyrical Bayes

In [388]:
np.random.seed(seed)

In [389]:
y_raw = data["wide"].copy()

In [390]:
# 최소 개수 제한
obs = y_raw.notna().sum()
y = y_raw.loc[:, obs[obs >= min_obs].index]
n_fcts = len(y.columns)

In [391]:
y_mean = y.mean()

In [392]:
if sigma is None:
    if bs_cov:
        bs_samples_list = []
        for i in tqdm(range(bs_samples)):
            # 행을 중복을 허용하면서 sample하면서 mean을 계산
            sample = resample(y, replace=True)
            bs_samples_list.append(sample.mean())

        bs_full = pd.DataFrame(bs_samples_list)
        bs_full_cov = bs_full.cov()

        alpha_sd = pd.Series(np.sqrt(np.diag(bs_full_cov)), index=y_mean.index)
        alpha_cor = bs_full.corr()
    else:
        y_sd = pd.Series(np.nanstd(y, axis=0), index=y.columns)
        alpha_sd = y_sd / np.sqrt(y.shape[0])
        alpha_cor = y.corr()

    alpha_cor_shrunk = (
        alpha_cor * (1 - shrinkage) + np.diag(np.full(n_fcts, 1)) * shrinkage
    )
    if cor_type == "sample":
        alpha_cor_adj = alpha_cor_shrunk
    elif cor_type == "block_clusters":
        alpha_cor_adj = block_cluster_func(alpha_cor_shrunk, cl_labels=cluster_labels)
    __corr = np.diag(alpha_sd) @ alpha_cor_adj @ np.diag(alpha_sd)
    sigma = pd.DataFrame(
        __corr.values, index=alpha_cor_adj.columns, columns=alpha_cor_adj.columns
    )
else:
    alpah_sd = np.sqrt(np.diag(sigma))

100%|██████████| 10000/10000 [00:19<00:00, 510.77it/s]


In [393]:
cm = y_mean.to_frame("value")
cm.index.name = "char_reg"
cm = cm.reset_index()
cm["characteristic"] = cm["char_reg"].str.split("__").str[0]
cm = cm.merge(cluster_labels, on="characteristic", how="left")

In [394]:
# Factor가 Cluster 어디에 포함돼 있는지 나타내는 matrix
m = (
    cm.assign(cm=1)[["char_reg", "hcl_label", "cm"]]
    .pivot(index="char_reg", columns="hcl_label", values="cm")
    .fillna(0)
    .copy()
)

mm = m @ m.T
# Cluster 개수
n_cl = m.shape[1]

**연습**

In [395]:
# 이와 같이 같은 Cluster면 함께면 sum axis=0이 포함되는 facor으 개수
tmp_m = np.array([[1, 0, 0], [1, 0, 0], [0, 0, 1]])
tmp_mm = tmp_m @ tmp_m.T
tmp_mm

array([[1, 1, 0],
       [1, 1, 0],
       [0, 0, 1]])

In [396]:
z = (
    cm.assign(sm=1)[["char_reg", "characteristic", "sm"]]
    .pivot(index="char_reg", columns="characteristic", values="sm")
    .fillna(0)
    .copy()
)
zz = z @ z.T
# 개별 factor의 개수
n_s = z.shape[1]

### Create Starting Value

In [397]:
starting_values = (
    cm.groupby(["hcl_label", "characteristic"])
    .agg(
        n_s=("value", "size"),  # Count of rows per group
        signal_mean=("value", "mean"),  # Mean of the 'value' column
        signal_sd=("value", "std"),  # Standard deviation of the 'value' column
    )
    .reset_index()
)

In [398]:
# region이 하나면 signal sd가 안나온다 (표본이 한개이므로)
starting_values

,hcl_label,characteristic,n_s,signal_mean,signal_sd
0,Accruals,capex_abn,3,0.177880,0.107352
1,Accruals,corr_1260d,3,0.387509,0.076620
2,Accruals,coskew_21d,3,0.453840,0.248294
3,Accruals,cowc_gr1a,3,0.468863,0.350190
4,Accruals,debt_gr3,3,0.464100,0.183163
...,...,...,...,...,...
148,Value,ami_126d,3,0.214952,0.036557
149,Value,market_equity,3,0.382444,0.211344
150,Value,ncol_gr1a,3,0.448165,0.152480
151,Value,oaccruals_at,3,0.417631,0.312261


In [399]:
starting_values = (
    starting_values.groupby("hcl_label")
    .agg(
        n_c=("n_s", "sum"),  # Sum of n_s within each cluster
        cl_mean=("signal_mean", "mean"),  # Mean of signal means
        cl_sd=("signal_mean", "std"),  # Standard deviation of signal means
        cl_signal_within=("signal_sd", "mean"),  # Mean of signal standard deviations
    )
    .reset_index()
)

In [400]:
# Cluster에 포함된 factor개수가 1개일때 cl_sd를 0으로 설정
starting_values["cl_sd"] = np.where(
    starting_values["n_c"] == 1, 0, starting_values["cl_sd"]
)

In [401]:
alpha_mean = starting_values["cl_mean"].mean()
print(alpha_mean)

0.3600318581235389


In [402]:
sd_cl_mean = (
    np.sqrt(np.sum(starting_values["cl_mean"] ** 2)) / (len(starting_values) - 1)
    if fix_alpha
    else starting_values["cl_mean"].std()
)
print(sd_cl_mean)

0.10880057627548773


In [403]:
sd_within_cl = starting_values["cl_sd"].mean()
print(sd_within_cl)

0.15399487126689365


In [404]:
sd_within_signal = starting_values["cl_signal_within"].mean()
print(sd_within_signal)

0.24967194296310471


In [405]:
starting_values = {
    "alpha_mean": alpha_mean,
    "sd_cl_mean": sd_cl_mean,
    "sd_within_cl": sd_within_cl,
    "sd_within_signal": sd_within_signal,
}

In [406]:
starting_values

{'alpha_mean': 0.3600318581235389,
 'sd_cl_mean': 0.10880057627548773,
 'sd_within_cl': 0.15399487126689365,
 'sd_within_signal': 0.24967194296310471}

## 전체 alpha adjust

In [407]:
if fix_alpha:
    sd_all = np.sqrt(y_mean.pow(2).sum()) / (len(y_mean) - 1)
else:
    sd_all = y_mean.std()

## Omega Function 정의

In [408]:
def omega_func(layers, tau_c, tau_s=None, tau_w=None):
    # Initialize the diagonal matrix based on the number of factors (n_fcts)
    if layers == 1:
        # All alphas are drawn from the same distribution
        a_omega = np.eye(n_fcts) * tau_c**2
    elif layers == 2:
        # All cluster alphas are drawn from the same distribution
        a_omega = np.eye(n_fcts) * tau_s**2 + mm * tau_c**2
    elif layers == 3:
        # Cluster distribution, signal distribution, and factor distribution
        a_omega = np.eye(n_fcts) * tau_w**2 + zz * tau_s**2 + mm * tau_c**2
    else:
        raise ValueError("layers should be 1, 2, or 3.")

    return a_omega

### MLE 되는 Parameter 찾기

In [409]:
if priors is None:
    if layers == 1:
        start_list = {
            'a': starting_values['alpha_mean'],
            'tc': sd_all
        }

        def mle_func(params):
            a, tc = params
            a_vec = np.full(n_fcts, a)
            a_omega = omega_func(layers=layers, tau_c=tc, tau_s=None, tau_w=None)
            a_cov = sigma + a_omega
            return -multivariate_normal.logpdf(y_mean, mean=a_vec, cov=a_cov)

    elif layers == 2:
        start_list = {
            'a': starting_values['alpha_mean'],
            'tc': starting_values['sd_cl_mean'],
            'ts': starting_values['sd_within_cl']
        }

        def mle_func(params):
            a, tc, ts = params
            a_vec = np.full(n_fcts, a)
            a_omega = omega_func(layers=layers, tau_c=tc, tau_s=ts, tau_w=None)
            a_cov = sigma + a_omega
            return -multivariate_normal.logpdf(y_mean, mean=a_vec, cov=a_cov)

    elif layers == 3:
        start_list = {
            'a': starting_values['alpha_mean'],
            'tc': starting_values['sd_cl_mean'],
            'ts': starting_values['sd_within_cl'],
            'tw': starting_values['sd_within_signal']
        }

        def mle_func(params):
            a, tc, ts, tw = params
            a_vec = np.full(n_fcts, a)
            a_omega = omega_func(layers=layers, tau_c=tc, tau_s=ts, tau_w=tw)
            a_cov = sigma + a_omega
            return -multivariate_normal.logpdf(y_mean, mean=a_vec, cov=a_cov)

    result = minimize(mle_func, list(start_list.values()), bounds=[(-np.inf, None)] + [(0, None)] * (len(start_list) - 1))
    assert result.success 
    # Extract final values
    mu = result.x[0]
    tau_c = result.x[1]
    tau_s = result.x[2] if layers > 1 else None
    tau_w = result.x[3] if layers == 3 else None
else:
    # Use priors
    mu = priors['alpha']
    tau_c = priors['tau_c']
    tau_s = priors['tau_s']
    tau_w = priors['tau_w']

## Calculate Theta

In [410]:
omega = omega_func(layers=layers, tau_c=tau_c, tau_s=tau_s, tau_w=tau_w)
print(f"Condition Number Omega = {np.round(np.linalg.cond(omega), 2)}")


Condition Number Omega = 1.0


In [411]:
omega

char_reg,age__developed,age__emerging,age__us,aliq_at__developed,aliq_at__emerging,aliq_at__us,aliq_mat__developed,aliq_mat__emerging,aliq_mat__us,ami_126d__developed,...,z_score__us,zero_trades_126d__developed,zero_trades_126d__emerging,zero_trades_126d__us,zero_trades_21d__developed,zero_trades_21d__emerging,zero_trades_21d__us,zero_trades_252d__developed,zero_trades_252d__emerging,zero_trades_252d__us
char_reg,,,,,,,,,,,,,,,,,,,,,
age__developed,5.949307e-02,7.738187e-14,7.738187e-14,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
age__emerging,7.738187e-14,5.949307e-02,7.738187e-14,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
age__us,7.738187e-14,7.738187e-14,5.949307e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
aliq_at__developed,0.000000e+00,0.000000e+00,0.000000e+00,5.949307e-02,7.738187e-14,7.738187e-14,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
aliq_at__emerging,0.000000e+00,0.000000e+00,0.000000e+00,7.738187e-14,5.949307e-02,7.738187e-14,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zero_trades_21d__emerging,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.738187e-14,5.949307e-02,7.738187e-14,0.000000e+00,0.000000e+00,0.000000e+00
zero_trades_21d__us,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.738187e-14,7.738187e-14,5.949307e-02,0.000000e+00,0.000000e+00,0.000000e+00
zero_trades_252d__developed,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,5.949307e-02,7.738187e-14,7.738187e-14


In [465]:
if layers == 3:
    # Error!! Cov에 대각행렬이 - 나옴
    z_transpose = z.T  # Transpose of matrix z
    theta_sigma_inv = np.linalg.inv(omega + sigma)
    as_mean = tau_w**2 * z_transpose @ theta_sigma_inv @ (y_mean - np.full(n_fcts, mu)).values
    as_cov = tau_w**2 * np.eye(n_s) - tau_w**4 * z_transpose @ theta_sigma_inv @ z.values 
    # as_cov.columns = as_cov.index
    # as_sd = np.sqrt(np.diag(as_cov))

    # # Convert as_mean and as_sd to DataFrame
    # as_mean_df = pd.DataFrame(as_mean, index=[z.columns], columns=["post_mean"])
    # as_sd_df = pd.DataFrame(as_sd, index=[z.columns], columns=["post_sd"])

    # # Merge as_mean and as_sd into signal_summary
    # signal_summary = pd.merge(as_mean_df.reset_index(), as_sd_df.reset_index(), on="index", how="left")
    # signal_summary.rename(columns={"index": "characteristic"}, inplace=True)


**Factor Posteriors**

In [ ]:
omega_inv = np.linalg.inv(omega)
sigma_inv = np.linalg.inv(sigma)

ai_cov is the posterior covariance matrix for factors

In [ ]:
ai_cov = np.linalg.inv(omega_inv + sigma_inv)
ai_sd = np.sqrt(np.diag(ai_cov))

In [ ]:
ai_mean = ai_cov @ (omega_inv @ np.full(n_fcts, mu) + sigma_inv @ y_mean)
ai_mean = pd.Series(ai_mean, index=y_mean.index)
ai_sd = pd.Series(ai_sd, index=y_mean.index)
ai_cov = pd.DataFrame(ai_cov, index=y_mean.index, columns=y_mean.index)

## Factor Summary

In [ ]:
factor_summary = pd.concat([
    ai_mean.to_frame("post_mean"), 
    ai_sd.to_frame("post_sd"), 
    y_mean.to_frame("osl_est"), 
    alpha_sd.to_frame("osl_se"), 
], axis=1).reset_index()

In [ ]:
factor_summary

,char_reg,post_mean,post_sd,osl_est,osl_se
0,age__us,-0.051492,0.063759,-0.153475,0.086150
1,aliq_at__us,0.212248,0.073889,0.312448,0.100315
2,aliq_mat__us,-0.242218,0.083257,-0.352743,0.099547
3,ami_126d__us,0.109482,0.075411,0.050272,0.085352
4,at_be__us,-0.038092,0.073495,-0.158214,0.100408
...,...,...,...,...,...
148,turnover_var_126d__us,-0.164078,0.069507,-0.128605,0.085679
149,z_score__us,0.113111,0.079154,-0.003803,0.108948
150,zero_trades_126d__us,0.284241,0.063804,0.376804,0.085989
151,zero_trades_21d__us,0.107779,0.063698,0.189812,0.085837


In [ ]:
factor_summary["characteristic"] = factor_summary["char_reg"].str.split("__").str[0]
factor_summary["p025"] = factor_summary["post_mean"] - 1.96 * factor_summary["post_sd"]
factor_summary["p975"] = factor_summary["post_mean"] + 1.96 * factor_summary["post_sd"]

In [ ]:
# Merget Cluster Label
factor_summary = factor_summary.merge(cluster_labels, on="characteristic", how="left")

In [ ]:
factor_summary

,char_reg,post_mean,post_sd,osl_est,osl_se,characteristic,p025,p975,hcl_label
0,age__us,-0.051492,0.063759,-0.153475,0.086150,age,-0.176461,0.073477,Low Leverage
1,aliq_at__us,0.212248,0.073889,0.312448,0.100315,aliq_at,0.067425,0.357070,Investment
2,aliq_mat__us,-0.242218,0.083257,-0.352743,0.099547,aliq_mat,-0.405401,-0.079034,Size
3,ami_126d__us,0.109482,0.075411,0.050272,0.085352,ami_126d,-0.038324,0.257287,Value
4,at_be__us,-0.038092,0.073495,-0.158214,0.100408,at_be,-0.182142,0.105959,Low Leverage
...,...,...,...,...,...,...,...,...,...
148,turnover_var_126d__us,-0.164078,0.069507,-0.128605,0.085679,turnover_var_126d,-0.300312,-0.027844,Profit Growth
149,z_score__us,0.113111,0.079154,-0.003803,0.108948,z_score,-0.042031,0.268253,Low Leverage
150,zero_trades_126d__us,0.284241,0.063804,0.376804,0.085989,zero_trades_126d,0.159185,0.409297,Seasonality
151,zero_trades_21d__us,0.107779,0.063698,0.189812,0.085837,zero_trades_21d,-0.017069,0.232626,Seasonality


In [ ]:
factor_summary["region"] = factor_summary["char_reg"].str.extract(r"__(.*)")

In [ ]:
factor_summary

,char_reg,post_mean,post_sd,osl_est,osl_se,characteristic,p025,p975,hcl_label,region
0,age__us,-0.051492,0.063759,-0.153475,0.086150,age,-0.176461,0.073477,Low Leverage,us
1,aliq_at__us,0.212248,0.073889,0.312448,0.100315,aliq_at,0.067425,0.357070,Investment,us
2,aliq_mat__us,-0.242218,0.083257,-0.352743,0.099547,aliq_mat,-0.405401,-0.079034,Size,us
3,ami_126d__us,0.109482,0.075411,0.050272,0.085352,ami_126d,-0.038324,0.257287,Value,us
4,at_be__us,-0.038092,0.073495,-0.158214,0.100408,at_be,-0.182142,0.105959,Low Leverage,us
...,...,...,...,...,...,...,...,...,...,...
148,turnover_var_126d__us,-0.164078,0.069507,-0.128605,0.085679,turnover_var_126d,-0.300312,-0.027844,Profit Growth,us
149,z_score__us,0.113111,0.079154,-0.003803,0.108948,z_score,-0.042031,0.268253,Low Leverage,us
150,zero_trades_126d__us,0.284241,0.063804,0.376804,0.085989,zero_trades_126d,0.159185,0.409297,Seasonality,us
151,zero_trades_21d__us,0.107779,0.063698,0.189812,0.085837,zero_trades_21d,-0.017069,0.232626,Seasonality,us


In [ ]:
if priors is None:
    comparison = pd.DataFrame({
        "estimate": ["alpha", "tau_c", "tau_s", "tau_w"][:layers + 1],
        "crude": pd.Series(start_list).values,
        "ml_est": [mu, tau_c, tau_s, tau_w][:layers + 1]
    })



In [ ]:
ret_list = {
    "input": data,
    "factors": factor_summary,
    "factor_mean": ai_mean,
    "factor_cov": ai_cov,
    "theta": omega,
    "sigma": sigma
}


In [ ]:
if sigma is None:
    ret_list["alpha_cor_raw"] = alpha_cor_shrunk
    ret_list["alpha_cor_adj"] = alpha_cor_adj

if priors is None:
    ret_list["mle"] = comparison

if layers == 3:
    ret_list["signal"] = signal_summary

In [ ]:
ret_list.keys()

dict_keys(['input', 'factors', 'factor_mean', 'factor_cov', 'theta', 'sigma', 'mle'])